In [82]:
from grab_util import *
import json
import pandas as pd
import numpy as np
import math
import urllib
import requests
from requests import ConnectionError, ReadTimeout

In [83]:
class Request(object):
    def __init__(self, request_session):
        self.request_session = request_session
    
    def get(self, url, headers={'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }, timeout=10, allow_redirects=False):
        try:
            print(url)
            headers['Host'] = urllib.request.splithost(urllib.request.splittype(url)[1])[0]
            res = self.request_session.get(url, headers=headers, timeout=timeout, allow_redirects=allow_redirects)
        except ConnectionError as ce:
            print('ConnectionError: ' + str(ce))
            return self.get(self, url=url, headers=headers, timeout=timeout, allow_redirects=allow_redirects)
        except ReadTimeout as rte:
            print('ReadTimeout: ' + str(rte))
            return self.get(self, url=url, headers=headers, timeout=timeout, allow_redirects=allow_redirects)

        return res
    
    def post(self, url, data, headers={'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }, timeout=10, allow_redirects=False):
        try:
            headers['host'] = urllib.request.splithost(urllib.request.splittype(url)[1])[0]
            res = self.request_session.post(url, data=data, headers=headers, timeout=timeout, allow_redirects=allow_redirects)
        except ConnectionError as ce:
            print('ConnectionError: ' + str(ce))
            return self.post(self, url=url, data=data, headers=headers, timeout=timeout, allow_redirects=allow_redirects)
        except ReadTimeout as rte:
            print('ReadTimeout: ' + str(rte))
            return self.post(self, url=url, data=data, headers=headers, timeout=timeout, allow_redirects=allow_redirects)

        return res

    def get_cookie_dict(self):
        return requests.utils.dict_from_cookiejar(self.request_session.cookies)

## Initial Cookies

In [84]:
url = "http://apps.webofknowledge.com/UA_AdvancedSearch.do"

headers = {
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.2 Safari/605.1.15",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Accept-Encoding': "gzip, deflate",
    'Accept-Language': "en-us",
    'Connection': "keep-alive"
    }

req = Request(requests.Session())
response = req.get(url, headers, 40, False)
print(response.headers)


while ('Location' in response.headers) and ('SID' not in response.headers.get('Location')):
    headers['Referer'] = url
    url = response.headers.get('Location')

    response = req.get(url, headers=headers, timeout=40, allow_redirects=False)
    print(response.headers)



http://apps.webofknowledge.com/UA_AdvancedSearch.do
{'Content-Length': '0', 'Server': 'Apache-Coyote/1.1', 'Location': 'http://www.webofknowledge.com?&Error=Client.NullSessionID', 'Date': 'Mon, 30 Dec 2019 09:37:29 GMT', 'Set-Cookie': 'JSESSIONID=3FEF758F916D8D3BD159956FCA419648; Path=/; HttpOnly, bm_sz=AE7F113F3CCB087BF4A856C8896D20BF~YAAQzawsFzBPzIVuAQAA9eMqVgZnO+3OanDa92JCWtPqR7JcWgiDeRzC/DfpkHIS/hU2a9fCHBQ09SNJBoKmLNjMoOIgrCNVAalFGei1q9R6UP/Z4fJ4lZAoUJttpUSHrU1suhlFjlBd132xofTn04CTuBXWDlJ1kX3dT+MPKbT+J0Sa0ZcupvWQxfP4GPPlPi48IFFW4O0=; Domain=.webofknowledge.com; Path=/; Expires=Mon, 30 Dec 2019 13:37:28 GMT; Max-Age=14399; HttpOnly, _abck=7D2E2C4C3CDC75B85BFDB585587B30AD~-1~YAAQzawsFzFPzIVuAQAA9eMqVgOVadItAclcE1EN7ZBK4zA3hpd5RC/Z1RDjiVQ86n/lJpdks4vG8ZG5el3Ro343XqVOmRRf3fD/zZghNoCPCCqFFMINqZrrMbZonE/AAP4Vtyt5Cmtf7Rsym3xxxbgqeUsmBcwc7KTru0UaFIJ3MDSKqw64FzSYg1LUWVYQ91kdUwBKdTbPJAu6kcyZUMzXRhE37BwzX1vgKLjBMipnfplvluy6UaOS9Ni5EsEu0nw0yMK2NdF8GnEg2WWmUOafb9lJE158GW2NF/I=~-1~-1~-1; Domain=

In [85]:
sid = req.get_cookie_dict()['SID']
req.get_cookie_dict()

{'CUSTOMER': '"SOUTH UNIV OF SCIENCE and TECH OF CHINA SUSTC"',
 'E_GROUP_NAME': '"SOUTH UNIV OF SCIENCE and TECH OF CHINA SUSTC"',
 'SID': '"6F42n2LeSaYxO3aINbl"',
 '_abck': '7D2E2C4C3CDC75B85BFDB585587B30AD~-1~YAAQzawsFzFPzIVuAQAA9eMqVgOVadItAclcE1EN7ZBK4zA3hpd5RC/Z1RDjiVQ86n/lJpdks4vG8ZG5el3Ro343XqVOmRRf3fD/zZghNoCPCCqFFMINqZrrMbZonE/AAP4Vtyt5Cmtf7Rsym3xxxbgqeUsmBcwc7KTru0UaFIJ3MDSKqw64FzSYg1LUWVYQ91kdUwBKdTbPJAu6kcyZUMzXRhE37BwzX1vgKLjBMipnfplvluy6UaOS9Ni5EsEu0nw0yMK2NdF8GnEg2WWmUOafb9lJE158GW2NF/I=~-1~-1~-1',
 'bm_sz': 'AE7F113F3CCB087BF4A856C8896D20BF~YAAQzawsFzBPzIVuAQAA9eMqVgZnO+3OanDa92JCWtPqR7JcWgiDeRzC/DfpkHIS/hU2a9fCHBQ09SNJBoKmLNjMoOIgrCNVAalFGei1q9R6UP/Z4fJ4lZAoUJttpUSHrU1suhlFjlBd132xofTn04CTuBXWDlJ1kX3dT+MPKbT+J0Sa0ZcupvWQxfP4GPPlPi48IFFW4O0=',
 'JSESSIONID': '3FEF758F916D8D3BD159956FCA419648'}

In [86]:
headers['Referer'] = url
url = response.headers.get('Location')
response = req.get(url, headers, 40, False)
print(response.headers)
headers['Referer'] = url
url = 'http://apps.webofknowledge.com/' + response.headers['Location']
response = req.get(url, headers, 40, False)
print(req.get_cookie_dict())

http://apps.webofknowledge.com/?Init=Yes&Error=Client.NullSessionID&SrcApp=CR&SID=6F42n2LeSaYxO3aINbl
{'Content-Type': 'text/html;charset=UTF-8', 'Content-Length': '0', 'Server': 'Apache-Coyote/1.1', 'Location': 'home.do?Init=Yes&Error=Client.NullSessionID&SrcApp=CR&SID=6F42n2LeSaYxO3aINbl', 'Date': 'Mon, 30 Dec 2019 09:37:30 GMT', 'Set-Cookie': 'JSESSIONID=54C123454DEDA5A2D75D6137797971D8; Path=/; HttpOnly, dotmatics.elementalKey=SLsLWlMhrHnTjDerSrlG; Version=1; Domain=.webofknowledge.com, ak_bmsc=B19FF4110178444EE29555740709703E172CACCDA57F00005AC5095EF901BA6B~plWngDBf/aOf3G083x+VvFmIwxWxeTyaUVSmKa2Y+WgD5GUYMaKpwLQWr5w5bde8UUOebYDHhe1DRMdi/SOfvB8dYe310zdf1sb7+zITzxr26PFzenVl8nIlfBMsJ3os0bd8xwjVG1oQLgElFshGt3Lg5v4RacqzN+Ye4cBetYPtyxtIt86Gu8ht4KMMVEPlWpoDB+xB3H1fdl4eAVrWCgaO/T52y5qcRLbTeGW2t3sx7z1Mln4gSjNWWPjQRtytE8; expires=Mon, 30 Dec 2019 11:37:30 GMT; max-age=7200; path=/; domain=.webofknowledge.com; HttpOnly', 'Connection': 'keep-alive'}
http://apps.webofknowledge.com/home.do?Init

In [87]:
sid_pure = sid.replace('"', '')

In [88]:
url = f'http://apps.webofknowledge.com/AutoSave_UA_AdvancedSearch_input.do?action=saveForm&SID={sid_pure}&product=UA&search_mode=AdvancedSearch'
search_data = [
    ('product', "UA"),
    ('search_mode', "AdvancedSearch"),
    ('SID', sid_pure),
    ('input_invalid_notice', "Search Error: Please enter a search term."),
    ('input_invalid_notice_limits', " <br/>Note: Fields displayed in scrolling boxes must be combined with at least one other search field."),
    ('action', "search"),
    ('replaceSetId', ""),
    ('goToPageLoc', "SearchHistoryTableBanner"),
    ('value(input1)', "SO=(OMEGA) AND PY=(2014 OR 2015 OR 2016 OR 2017 OR 2018)"),
    ('value(searchOp)', "search"),
    ('limitStatus', "expanded"),
    ('ss_lemmatization', "On"),
    ('ss_spellchecking', "Suggest"),
    ('SinceLastVisit_UTC', ""),
    ('SinceLastVisit_DATE', ""),
    ('period', "Range Selection"),
    ('range', "ALL"),
    ('startYear', "1900"),
    ('endYear', "2019"),
    ('editions', "WOS.SSCI"),
    ('editions', "WOS.SCI"),
    # ('editions', "WOS.IC"),
    # ('editions', "WOS.ISSHP"),
    # ('editions', "WOS.ISTP"),
    # ('editions', "WOS.CCR"),
    ('collections', "WOS"),
    # ('editions', "CSCD.CSCD"),
    # ('collections', "CSCD"),
    # ('editions', "SCIELO.SCIELO"),
    # ('collections', "SCIELO"),
    ('update_back2search_link_param', "yes"),
    ('ssStatus', "display,none"),
    ('ss_showsuggestions', "ON"),
    ('ss_query_language', "auto"),
    ('rs_sort_by', "PY.D;LD.D;SO.A;VL.D;PG.A;AU.A")
]

In [89]:
response = req.post(url, search_data, headers, 40, False)

headers['Referer'] = url
url = 'http://apps.webofknowledge.com/UA_AdvancedSearch.do'
response = req.post(url, search_data, headers, 40, False)
headers['Referer'] = url
url = 'http://apps.webofknowledge.com' + response.headers.get('Location')
response = req.get(url, headers=headers, timeout=40, allow_redirects=False)

http://apps.webofknowledge.com/UA_AdvancedSearch_input.do?product=UA&search_mode=AdvancedSearch&replaceSetId=&goToPageLoc=SearchHistoryTableBanner&SID=6F42n2LeSaYxO3aINbl&errorQid=1


In [90]:
start = response.text.find('<div id="set_1_div" class="historyResults">')
end = start + 500

In [91]:
response.text[start - 200:end]

'                                   <tr id="set_1_row" class=""> <td align="center" valign="top" class="historySetNum"> <div class="syntax33_highlight"># 1</div> </td> <td align="center" valign="top"> <div id="set_1_div" class="historyResults">                     <a href="/summary.do?product=UA&doc=1&qid=1&SID=6F42n2LeSaYxO3aINbl&search_mode=AdvancedSearch&update_back2search_link_param=yes" id="hitCount" class="hitCount" title="Click to view the results">641</a>            </div> </td> <td valign="top"> <div class="historyQuery">             <if test="true">   <if test="false">  </if>   <if test="false">  </if> </if>  SO=(OMEGA) AND PY=(2014 OR 2015 OR 2016 OR 2017 OR 2018)  </div> <div>    '

In [92]:
req.get_cookie_dict()

{'CUSTOMER': '"SOUTH UNIV OF SCIENCE and TECH OF CHINA SUSTC"',
 'E_GROUP_NAME': '"SOUTH UNIV OF SCIENCE and TECH OF CHINA SUSTC"',
 'SID': '"6F42n2LeSaYxO3aINbl"',
 '_abck': '7D2E2C4C3CDC75B85BFDB585587B30AD~-1~YAAQzawsFzFPzIVuAQAA9eMqVgOVadItAclcE1EN7ZBK4zA3hpd5RC/Z1RDjiVQ86n/lJpdks4vG8ZG5el3Ro343XqVOmRRf3fD/zZghNoCPCCqFFMINqZrrMbZonE/AAP4Vtyt5Cmtf7Rsym3xxxbgqeUsmBcwc7KTru0UaFIJ3MDSKqw64FzSYg1LUWVYQ91kdUwBKdTbPJAu6kcyZUMzXRhE37BwzX1vgKLjBMipnfplvluy6UaOS9Ni5EsEu0nw0yMK2NdF8GnEg2WWmUOafb9lJE158GW2NF/I=~-1~-1~-1',
 'ak_bmsc': 'B19FF4110178444EE29555740709703E172CACCDA57F00005AC5095EF901BA6B~plWngDBf/aOf3G083x+VvFmIwxWxeTyaUVSmKa2Y+WgD5GUYMaKpwLQWr5w5bde8UUOebYDHhe1DRMdi/SOfvB8dYe310zdf1sb7+zITzxr26PFzenVl8nIlfBMsJ3os0bd8xwjVG1oQLgElFshGt3Lg5v4RacqzN+Ye4cBetYPtyxtIt86Gu8ht4KMMVEPlWpoDB+xB3H1fdl4eAVrWCgaO/T52y5qcRLbTeGW2t3sx7z1Mln4gSjNWWPjQRtytE8',
 'bm_sv': '3566D1569B9F33F2345471B812F0E5C2~z3Zj/eNnZTGXs552DHuARSvcM4sgNltHU9Ja0MqT04jwk7gFlK7oB/NJ0O+c/lugIgzVuotbHkNLoodK9H8HPLlHlpWQ9YmbW

In [93]:
url = 'http://apps.webofknowledge.com' + '/summary.do?product=UA&doc=1&qid=1&SID=5COWRs8gVgnqzqpvhSj&search_mode=AdvancedSearch&update_back2search_link_param=yes'
response = req.get(url, headers=headers, timeout=40, allow_redirects=False)

http://apps.webofknowledge.com/summary.do?product=UA&doc=1&qid=1&SID=5COWRs8gVgnqzqpvhSj&search_mode=AdvancedSearch&update_back2search_link_param=yes
